In [3]:
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

# calculate adjusted R squared
# k = number of variables
# n = number of datapoints
def adjR2(r2, k, n):
    return 1- (((1 - r2) * (n - 1))/ (n - k - 1))

# calculate root mean square error
def rmse(mse):
    return sqrt(mse)

In [3]:
import pandas as pd
from sklearn.svm import LinearSVR
from sklearn.feature_selection import RFE
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from math import sqrt
import warnings

# Suppress ConvergenceWarning from sklearn
warnings.filterwarnings("ignore", category=UserWarning)

# Load the DataFrame
df = pd.read_csv("soilmoisture_dataset.csv", index_col=0)

# Get hyperspectral bands
hypbands = []
for col in df.columns:
    try:
        int(col)
    except Exception:
        continue
    hypbands.append(col)

# Prepare the data by dropping unnecessary columns
data = df.drop(['soil_temperature', 'datetime'], axis=1)

# Reset index to ensure sequential integer indices for compatibility with KFold
data = data.reset_index(drop=True)

# Define the adjusted R-squared function
def adjR2(r2, n_features, n_samples):
    return 1 - (1 - r2) * (n_samples - 1) / (n_samples - n_features - 1)

# Create a LinearSVR model
model = LinearSVR(C=250, random_state=42, dual="auto", max_iter=10000)

k = 15  # number of total features to compare

for n in range(k):
    # Create the RFE selector with the model
    rfe = RFE(model, n_features_to_select=n + 1)

    # Fit the RFE selector on the data
    fit = rfe.fit(data[hypbands], data["soil_moisture"])

    # Get the selected features as boolean array
    selected_features = fit.support_

    # Create a list of selected feature names
    selected_columns = [hypbands[i] for i in range(len(selected_features)) if selected_features[i]]
    print(f"Selected columns (iteration {n + 1}):", selected_columns)  # Debug print

    # Ensure all selected columns exist in data
    missing_columns = [col for col in selected_columns if col not in data.columns]
    if missing_columns:
        print(f"Missing columns in data for iteration {n + 1}: {missing_columns}")
        continue  # Skip iteration if columns are missing

    # Perform 5-fold cross-validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    rmse_list = []
    adjR_list = []

    for train_index, test_index in kf.split(data):
        # Split the data into training and testing sets
        X_train, X_test = data.loc[train_index, selected_columns], data.loc[test_index, selected_columns]
        y_train, y_test = data["soil_moisture"].iloc[train_index], data["soil_moisture"].iloc[test_index]

        # Scale the features
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)  # Fit and transform on training data
        X_test_scaled = scaler.transform(X_test)  # Transform the test data

        # Train the model on the scaled data
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)

        # Calculating mean squared error
        mse = mean_squared_error(y_test, y_pred)
        rmse_list.append(sqrt(mse))

        # Calculating adjusted R squared
        r2 = r2_score(y_test, y_pred)
        adjR_list.append(adjR2(r2, X_test.shape[1], X_test.shape[0]))

    # Average RMSE across folds
    average_rmse = sum(rmse_list) / len(rmse_list)
    average_adjR = sum(adjR_list) / len(adjR_list)

    # Print results
    print("Average RMSE:", round(average_rmse, 4), "Average Adjusted R-squared:", round(average_adjR, 4))


Selected columns (iteration 1): ['462']
Average RMSE: 2.7906 Average Adjusted R-squared: 0.4024
Selected columns (iteration 2): ['462', '950']
Average RMSE: 1.9756 Average Adjusted R-squared: 0.6935
Selected columns (iteration 3): ['462', '494', '950']
Average RMSE: 2.0016 Average Adjusted R-squared: 0.6831
Selected columns (iteration 4): ['462', '494', '498', '950']
Average RMSE: 1.9744 Average Adjusted R-squared: 0.6893
Selected columns (iteration 5): ['462', '494', '498', '766', '950']
Average RMSE: 1.8532 Average Adjusted R-squared: 0.7202
Selected columns (iteration 6): ['462', '490', '494', '498', '766', '950']
Average RMSE: 1.8506 Average Adjusted R-squared: 0.7194
Selected columns (iteration 7): ['462', '482', '490', '494', '498', '766', '950']
Average RMSE: 1.8418 Average Adjusted R-squared: 0.72
Selected columns (iteration 8): ['462', '482', '490', '494', '498', '762', '766', '950']
Average RMSE: 1.8583 Average Adjusted R-squared: 0.7131
Selected columns (iteration 9): ['462'

In [6]:
import pandas as pd
from sklearn.svm import LinearSVR
from sklearn.svm import SVR
from sklearn.feature_selection import RFE
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from math import sqrt
import warnings

# Suppress ConvergenceWarning from sklearn
warnings.filterwarnings("ignore", category=UserWarning)

# Load the DataFrame
df = pd.read_csv("soilmoisture_dataset.csv", index_col=0)

# Get hyperspectral bands
hypbands = []
for col in df.columns:
    try:
        int(col)
    except Exception:
        continue
    hypbands.append(col)

# Prepare the data by dropping unnecessary columns
data = df.drop(['soil_temperature', 'datetime'], axis=1)

# Reset index to ensure sequential integer indices for compatibility with KFold
data = data.reset_index(drop=True)

# Define the adjusted R-squared function
def adjR2(r2, n_features, n_samples):
    return 1 - (1 - r2) * (n_samples - 1) / (n_samples - n_features - 1)

# Create a LinearSVR model
model = LinearSVR(C=250, random_state=42, dual="auto", max_iter=10000)
model2 = SVR(kernel='rbf', C=1000, gamma = 'scale')

k = 15  # number of total features to compare

for n in range(k):
    # Create the RFE selector with the model
    rfe = RFE(model, n_features_to_select=n + 1)

    # Fit the RFE selector on the data
    fit = rfe.fit(data[hypbands], data["soil_moisture"])

    # Get the selected features as boolean array
    selected_features = fit.support_

    # Create a list of selected feature names
    selected_columns = [hypbands[i] for i in range(len(selected_features)) if selected_features[i]]
    print(f"Selected columns (iteration {n + 1}):", selected_columns)  # Debug print

    # Ensure all selected columns exist in data
    missing_columns = [col for col in selected_columns if col not in data.columns]
    if missing_columns:
        print(f"Missing columns in data for iteration {n + 1}: {missing_columns}")
        continue  # Skip iteration if columns are missing

    # Perform 5-fold cross-validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    rmse_list = []
    adjR_list = []

    for train_index, test_index in kf.split(data):
        # Split the data into training and testing sets
        X_train, X_test = data.loc[train_index, selected_columns], data.loc[test_index, selected_columns]
        y_train, y_test = data["soil_moisture"].iloc[train_index], data["soil_moisture"].iloc[test_index]

        # Scale the features
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)  # Fit and transform on training data
        X_test_scaled = scaler.transform(X_test)  # Transform the test data

        # Train the model on the scaled data
        model2.fit(X_train_scaled, y_train)
        y_pred = model2.predict(X_test_scaled)

        # Calculating mean squared error
        mse = mean_squared_error(y_test, y_pred)
        rmse_list.append(sqrt(mse))

        # Calculating adjusted R squared
        r2 = r2_score(y_test, y_pred)
        adjR_list.append(adjR2(r2, X_test.shape[1], X_test.shape[0]))

    # Average RMSE across folds
    average_rmse = sum(rmse_list) / len(rmse_list)
    average_adjR = sum(adjR_list) / len(adjR_list)

    # Print results
    print("Average RMSE:", round(average_rmse, 4), "Average Adjusted R-squared:", round(average_adjR, 4))


Selected columns (iteration 1): ['462']
Average RMSE: 2.5976 Average Adjusted R-squared: 0.4797
Selected columns (iteration 2): ['462', '950']
Average RMSE: 1.4098 Average Adjusted R-squared: 0.8457
Selected columns (iteration 3): ['462', '494', '950']
Average RMSE: 1.3706 Average Adjusted R-squared: 0.8522
Selected columns (iteration 4): ['462', '494', '498', '950']
Average RMSE: 1.3671 Average Adjusted R-squared: 0.8521
Selected columns (iteration 5): ['462', '494', '498', '766', '950']
Average RMSE: 1.0845 Average Adjusted R-squared: 0.9058
Selected columns (iteration 6): ['462', '490', '494', '498', '766', '950']
Average RMSE: 1.1322 Average Adjusted R-squared: 0.8962
Selected columns (iteration 7): ['462', '482', '490', '494', '498', '766', '950']
Average RMSE: 1.1228 Average Adjusted R-squared: 0.8977
Selected columns (iteration 8): ['462', '482', '490', '494', '498', '762', '766', '950']
Average RMSE: 1.109 Average Adjusted R-squared: 0.8989
Selected columns (iteration 9): ['462